In [1]:
from data_prep import *
from charts import *
from video_analysis import *
from team_sheets import *

# Load data
game_df = team_sheets()
players_df = players(game_df)
players_agg_df = players_agg(players_df)
lineouts_df = lineouts()
set_piece_df = set_piece_results()
analysis = game_stats()
 
# # Save data
# game_df.to_csv('data/game.csv', index=False)
# players_df.to_csv('data/players.csv', index=False)
# players_agg_df.to_csv('data/players_agg.csv', index=False)
# lineouts_df.to_csv('data/lineouts.csv', index=False)
# set_piece_df.to_csv('data/set_piece.csv', index=False)
# analysis.to_csv('data/analysis.csv', index=False)
# update_season_summaries(game_df, seasons=["2024/25"])

# # One-off charts (only source data needs updating)
# captains_chart(file='Charts/captains.html')
# results_chart(file='Charts/results.html')
# plot_games_by_player(file='Charts/appearances.html')
# plot_starts_by_position(file='Charts/positions.html')
# card_chart(file='Charts/cards.html')
# points_scorers_chart(file='Charts/points.html')
# team_sheets_chart(file='Charts/team-sheets.html')
# set_piece_h2h_chart(file='Charts/set-piece.html')
# squad_continuity_chart(file='Charts/continuity.html')

# # Self-contained charts (chart needs updating)
# game_stats_charts(analysis, file='Charts/video_analysis.html')
# lineout_success(types=types, file='Charts/lineouts.html')

In [2]:
team_sheet_part(game_df, "Forwards")

NameError: name 'team_sheet_part' is not defined

In [69]:
players_df

,GameID,Squad,Season,Competition,GameType,Opposition,Home/Away,PF,PA,Result,Captain,VC1,VC2,Number,Player,Position,PositionType
0,Crawley (H),1st,2021/22,Friendly,Friendly,Crawley,H,34,12,W,Jack Andrews,James Funnell,<NA>,1,James Funnell,Prop,Forwards
1,University of Brighton (A),1st,2021/22,Friendly,Friendly,University of Brighton,A,0,43,L,Jack Andrews,James Funnell,<NA>,1,James Funnell,Prop,Forwards
2,Metropolitan Police (H),1st,2021/22,Friendly,Friendly,Metropolitan Police,H,29,28,W,Jack Andrews,<NA>,<NA>,1,James Funnell,Prop,Forwards
3,London Irish (A),1st,2021/22,Friendly,Friendly,London Irish,A,10,33,L,Jack Andrews,Sam Lindsay-McCall,<NA>,1,Sean Morgan,Prop,Forwards
4,Horsham (A),1st,2021/22,Friendly,Friendly,Horsham,A,26,56,L,Jack Andrews,James Funnell,<NA>,1,James Funnell,Prop,Forwards
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3482,Metropolitan Police (H),1st,2021/22,Friendly,Friendly,Metropolitan Police,H,29,28,W,Jack Andrews,<NA>,<NA>,25,Henry Weller,NaN,Bench
3520,Old Colfeians (H),1st,2023/24,Friendly,Friendly,Old Colfeians,H,19,36,L,Connor Leggat,Oli Billin,Sam Lindsay-McCall,25,Ben Beard,NaN,Bench
3521,Heathfield & Waldron (H),1st,2023/24,Friendly,Friendly,Heathfield & Waldron,H,0,5,L,Max Crawley-Moore,<NA>,<NA>,25,Colm Geraghty,NaN,Bench
3603,Crowborough (H),2nd,2023/24,Counties 4 Sussex,League,Crowborough,H,10,7,W,Ben Watkinson,Max Crawley-Moore,<NA>,25,Jack Andrews,NaN,Bench


,Player,Position
0,Aaron Boczek,Number 8 / Flanker
1,Alex Jackson,Number 8
2,Alex Leadbeater,Centre / Wing
3,Alex Maynard,Prop
4,Ali Moffatt,Wing / Centre
...,...,...
123,Tom Marr,Fly Half
124,Tom Mooney,Flanker
125,Will Bramwell,Second Row / Flanker
126,Will Burch,Prop


In [85]:
players_agg_df.loc[:,"MostCommonPosition":]

,MostCommonPosition,MostCommonPositionType,Player_join,A,T,Con,PK,DG,YC,RC,Points,PPG,Tries,Cons,Pens
0,Prop,Forwards,D Bridges,15.0,4.0,0.0,0.0,0.0,1.0,0.0,20.0,1.333333,20.0,0.0,0.0
1,Prop,Forwards,J Funnell,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
2,Prop,Forwards,S Morgan,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
3,Prop,Forwards,B Pellett,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
4,Prop,Forwards,B Watkinson,11.0,1.0,0.0,0.0,0.0,1.0,0.0,5.0,0.454545,5.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
532,NaN,None,R Mcgealy,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
533,NaN,None,S Lampkin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
534,NaN,None,D King,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
535,NaN,None,S Spriddell,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0


# Individual Player Stats

- Table:
  - Position
  - Debut
  - Total Games (this season/total)
  - Tries (this season/total)
- Charts:
  - Games per season (color by squad)
  - Games by position (color by squad)
  - Tries/points per season

In [107]:
current_season = max(players_agg_df["Season"])

# Position dictionary
d = {
    1: "Prop",
    2: "Hooker",
    3: "Prop",
    4: "Second Row",
    5: "Second Row",
    6: "Flanker",
    7: "Flanker",
    8: "Number 8",
    9: "Scrum Half",
    10: "Fly Half",
    11: "Wing",
    12: "Centre",
    13: "Centre",
    14: "Wing",
    15: "Full Back",
}


# Total tries per player (current season)
def totals(df):
    totals = (
        df.groupby("Player")
        .agg({"T": "sum", "TotalGames": "sum"})
        .rename(columns={"TotalGames": "Games", "T": "Tries"})
        .reset_index()
        .astype({"Tries": "int", "Games": "int"})
    )

    games_by_squad = (
        df.groupby(["Player", "Squad"])
        .agg({"TotalGames": "sum"})
        .reset_index()
        .pivot(index="Player", columns="Squad", values="TotalGames")
        .reset_index()
        .rename(columns={"1st": "Games1", "2nd": "Games2"})
        .fillna(0)
        .astype({"Games1": "int", "Games2": "int"})
    )

    totals = totals.merge(games_by_squad, on="Player").fillna(0)

    return totals

def get_positions(df, by=None):

    df["Position"] = df.apply(lambda x: d.get(x["Number"]), axis=1)


    df = (
        df.groupby(["Player", "Position", by] if by else ["Player", "Position"])
        .agg({"PF": "count"})
        .reset_index()
        .sort_values(["Player", "PF"], ascending=[True, False])
    )

    df = df[df['PF'] >= 1]

    return df


def players_table_data():

    positions = get_positions(players_df)
    positions = positions[
        positions.groupby('Player')['PF'].transform('sum') * 0.2 <= positions[~positions['Position'].isna()]['PF']
    ]
    positions = positions.groupby('Player').agg({'Position': lambda x: ' / '.join(x)}).reset_index()

    df_current = totals(players_agg_df[players_agg_df["Season"] == current_season])
    df_total = totals(players_agg_df).rename(columns={
        "Tries": "TotalTries", 
        "Games": "TotalGames", 
        "Games1": "TotalGames1", 
        "Games2": "TotalGames2"
    })

    df = (
        df_total
        .merge(df_current, on="Player", how="left").fillna(0)
        .merge(positions, on="Player", how="left")
        .astype({"Tries": "int", "Games": "int", "Games1": "int", "Games2": "int"})
    )

    df.to_json("data/player_table.json", orient="records", indent=2)

    return df

df = players_table_data()
df


,Player,TotalTries,TotalGames,TotalGames1,TotalGames2,Tries,Games,Games1,Games2,Position
0,Aaron Boczek,10,40,37,3,0,4,1,3,Number 8 / Flanker
1,Aaron Stonell,0,1,0,1,0,0,0,0,Prop
2,Alex Jackson,2,5,4,1,0,0,0,0,Number 8
3,Alex Leadbeater,2,9,1,8,0,0,0,0,Centre / Wing
4,Alex Maynard,2,2,0,2,0,0,0,0,Prop
...,...,...,...,...,...,...,...,...,...,...
186,Tommy Nasta,0,1,0,1,0,1,0,1,Wing
187,Will Bramwell,0,10,4,6,0,3,1,2,Second Row / Flanker
188,Will Burch,0,4,0,4,0,0,0,0,Prop / Hooker
189,Will Roberts,29,38,10,28,6,13,6,7,Wing


In [109]:
df[df["Player"] == "Colm Geraghty"]
# get_positions(players_df)[get_positions(players_df)["Player"] == "Colm Geraghty"]

,Player,TotalTries,TotalGames,TotalGames1,TotalGames2,Tries,Games,Games1,Games2,Position
31,Colm Geraghty,5,9,7,2,3,3,1,2,Centre / Full Back


In [ ]:
p = "Dan Billin"

def squad_pie(p):
    base = (
        alt.Chart(players_agg_df).encode(
            theta=alt.Theta("sum(TotalGames)").stack(True),
            color=alt.Color("Squad:N", scale=squad_scale, legend=alt.Legend(title=None, labelExpr="datum.label + ' XV'"))
        )
        .transform_filter(f"datum.Player === '{p}'")
        .transform_calculate(label="datum.Squad + ' XV'")
    )

    pie = base.mark_arc(outerRadius=120, opacity=0.8)
    text1 = base.mark_text(radius=75, size=36).encode(
        theta=alt.Theta("sum(TotalGames)", stack=True),
        text=alt.Text("sum(TotalGames)"), 
        detail="Squad:N",
        color=alt.value("white")
    )
    text2 = base.mark_text(radius=150, size=24).encode(
        theta=alt.Theta("sum(TotalGames)", stack=True),
        text=alt.Text("label:N"),
        detail="Squad:N",
    )

    return pie + text1 + text2

position_order = ["Prop", "Hooker", "Second Row", "Back Row", "Scrum Half", "Fly Half", "Centre", "Back Three"]
position_color = ["#202947", "#146f14", "#981515", "#b03030"]

def position_pie(p):
    base = (
        alt.Chart(players_df)
        .transform_calculate(posi=f"indexof({position_order}, datum.Position)")
        .encode(
            theta=alt.Theta("count()").stack(True),
            color=alt.Color(
                "Position:N"
                legend=alt.Legend(title=None, orient="bottom", offset=40), 
                scale=alt.Scale()
            )
        )
        .transform_filter(f"datum.Player === '{p}' & isValid(datum.Position)")
    )

    pie = base.mark_arc(outerRadius=120)
    text = base.mark_text(radius=75, size=36).encode(
        theta=alt.Theta("count()", stack=True),
        text=alt.Text("count()"), 
        color=alt.value("white"),
        detail="Position:N"
    )

    return (pie + text).transform_filter(f"datum.Player === '{p}'")

def games(p):
    bar = (
        alt.Chart(players_agg_df).encode(
            x=alt.X("Date:T", title="Date"),
            y=alt.Y("count()", title="Games Played"),
            color=alt.Color("Position:N", scale=position_scale)
        )
        .transform_filter(f"datum.Player === '{p}'")
        .mark_bar()
    )

position_pie(p)
# squad_pie(p)


SyntaxError: invalid syntax. Perhaps you forgot a comma? (2585547723.py, line 38)

In [ ]:
b

,Squad,Season,Competition,GameType,Opposition,Home/Away,PF,PA,Result,Captain,VC1,VC2,Number,Player,Position,PositionType
422,1st,2021/22,Friendly,Friendly,Metropolitan Police,H,29,28,W,Jack Andrews,<NA>,<NA>,4,Sam Lindsay-McCall,Second Row,Forwards
423,1st,2021/22,Friendly,Friendly,London Irish,A,10,33,L,Jack Andrews,Sam Lindsay-McCall,<NA>,4,Sam Lindsay-McCall,Second Row,Forwards
424,1st,2021/22,Friendly,Friendly,Horsham,A,26,56,L,Jack Andrews,James Funnell,<NA>,4,Sam Lindsay-McCall,Second Row,Forwards
425,1st,2021/22,Friendly,Friendly,Purley John Fisher,H,19,33,L,Jack Andrews,James Funnell,<NA>,4,Sam Lindsay-McCall,Second Row,Forwards
426,1st,2021/22,Sussex 1,League,Eastbourne,H,47,0,W,Jack Andrews,<NA>,<NA>,4,Sam Lindsay-McCall,Second Row,Forwards
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2289,1st,2023/24,Counties 1 Surrey/Sussex,League,Eastbourne,A,24,26,L,Ryland Thomas,Dan Poulton,<NA>,17,Sam Lindsay-McCall,NaN,Bench
2313,1st,2024/25,Counties 1 Surrey/Sussex,League,Eastbourne,A,36,33,W,Ryland Thomas,Chris May-Miller,<NA>,17,Sam Lindsay-McCall,NaN,Bench
2314,1st,2024/25,Counties 1 Surrey/Sussex,League,Trinity,H,14,48,L,Ryland Thomas,<NA>,<NA>,17,Sam Lindsay-McCall,NaN,Bench
2315,1st,2024/25,Counties 1 Surrey/Sussex,League,Old Rutlishians,H,19,23,L,Ryland Thomas,<NA>,<NA>,17,Sam Lindsay-McCall,NaN,Bench


# Team Sheets

In [ ]:
def team_sheet_chart(
        squad=1, 
        names=None, 
        captain=None, 
        vc=None, 
        opposition=None, 
        home=True, 
        competition="Counties 1 Sussex",
        season="2023/24"
    ):

    if names is None:
        df = team_sheets()    

        # Last row as dict
        team = df.iloc[-1].to_dict()


        label = f'{"1st" if squad==1 else "2nd"} XV vs {team["Opposition"]}({team["Home/Away"]})'
        captain = team["Captain"]
        vc = team["VC1"]
        season = team["Season"]
        competition = team["Competition"]

        # Keep keys that can be converted to integers
        team = {int(k): v for k, v in team.items() if k.isnumeric() and v}

        # Convert team to dataframe with Number and Player columns
        team = pd.DataFrame(team.items(), columns=["Number", "Player"])

    else:
        label = f'{"1st" if squad==1 else "2nd"} XV vs {opposition} ({"H" if home else "A"})'

        # Convert names to Player column of a dataframe with Number column (1-len(names))
        team = pd.DataFrame({"Player": names, "Number": range(1, len(names)+1)})

    coords = pd.DataFrame([
                {"n": 1, "x": 10, "y": 81},
                {"n": 2, "x": 25, "y": 81},
                {"n": 3, "x": 40, "y": 81},
                {"n": 4, "x": 18, "y": 69},
                {"n": 5, "x": 32, "y": 69},
                {"n": 6, "x": 6, "y": 61},
                {"n": 7, "x": 44, "y": 61},
                {"n": 8, "x": 25, "y": 56},
                {"n": 9, "x": 20, "y": 42},
                {"n": 10, "x": 38, "y": 36},
                {"n": 11, "x": 8, "y": 18},
                {"n": 12, "x": 56, "y": 30},
                {"n": 13, "x": 74, "y": 24},
                {"n": 14, "x": 92, "y": 18},
                {"n": 15, "x": 50, "y": 10},
                {"n": 16, "x": 80, "y": 82},
                {"n": 17, "x": 80, "y": 74},
                {"n": 18, "x": 80, "y": 66},
                {"n": 19, "x": 80, "y": 58},
                {"n": 20, "x": 80, "y": 50},
                {"n": 21, "x": 80, "y": 42},
                {"n": 22, "x": 80, "y": 34},
                {"n": 23, "x": 80, "y": 26},
            ])
    team = team.merge(coords, left_on="Number", right_on="n", how="inner").drop(columns="n")

    # Add captain (C) and vice captain (VC) else None
    team["Captain"] = team["Player"].apply(lambda x: "C" if x == captain else "VC" if x == vc else None)

    team["Player"] = team["Player"].str.split(" ")

    team.to_dict(orient="records")

    with open("team-sheet-lineup.json") as f:
        chart = json.load(f)
    chart["data"]["values"] = team.to_dict(orient="records")
    chart["title"]["text"] = label
    chart["title"]["subtitle"] = f"{season} - {competition}"

    n_replacements = len(team) - 15
    
    y = 126 + (n_replacements * 64)
    chart["layer"][0]["mark"]["y2"] = y
    # return chart
    return alt.Chart.from_dict(chart)

team_sheet_chart()

alt.LayerChart(...)

In [ ]:
pitchero_df[pitchero_df["Player"]=="Sam Lindsay-McCall"]

,Player,Season,Squad,TotalGames,Player_join,A,T,Con,PK,DG,YC,RC,Points,PPG,Tries,Cons,Pens,Cards
9,Sam Lindsay-McCall,2023/24,1st,23,S Lindsay-Mccall,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,Sam Lindsay-McCall,2021/22,2nd,2,S Lindsay-Mccall,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51,Sam Lindsay-McCall,2024/25,2nd,1,S Lindsay-Mccall,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
198,Sam Lindsay-McCall,2021/22,1st,15,S Lindsay-Mccall,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
200,Sam Lindsay-McCall,2024/25,1st,15,S Lindsay-Mccall,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
270,Sam Lindsay-McCall,2022/23,2nd,1,S Lindsay-Mccall,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
470,Sam Lindsay-McCall,2022/23,1st,15,S Lindsay-Mccall,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
pitchero_df[pitchero_df["Player_join"]=="S Lindsay"].sort_values(["Season","Squad"])

,Player,Season,Squad,TotalGames,Player_join,A,T,Con,PK,DG,YC,RC,Points,PPG,Tries,Cons,Pens
464,Sam Lindsay-McCall,2021/22,1st,15,S Lindsay,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
348,Sam Lindsay-McCall,2021/22,2nd,2,S Lindsay,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,Sam Lindsay-McCall,2022/23,1st,15,S Lindsay,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
405,Sam Lindsay-McCall,2022/23,2nd,1,S Lindsay,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
346,Sam Lindsay-McCall,2023/24,1st,23,S Lindsay,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
466,Sam Lindsay-McCall,2024/25,1st,15,S Lindsay,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
388,Sam Lindsay-McCall,2024/25,2nd,1,S Lindsay,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
